In [1]:
import ee
from datetime import datetime
from dotenv import load_dotenv
import geemap
import geopandas as gpd
import os

In [2]:
EE_PROJECT_ID = os.environ.get("EE_PROJECT_ID")

In [3]:
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID)

In [12]:
aoi = gpd.read_file("../data/adm_2/lao_admbnda_adm2_ngd_20191112.shp").explode()
aoi_geometry = aoi[aoi["ADM2_EN"] == "Chomphet"]["geometry"].iloc[0]
coords = [list(aoi_geometry.exterior.coords)]
aoi_gee = ee.Geometry.Polygon(coords)

In [7]:
# Applies scaling factors.
def apply_scale_factors_l5_l7(image):
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermal_bands = image.select("ST_B6").multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)


def add_evi_l5_l7(image):
    # Landsat 5,7: B3(Red), B4(NIR), B1(Blue)
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
        {
            "NIR": image.select("SR_B4"),
            "RED": image.select("SR_B3"),
            "BLUE": image.select("SR_B1"),
        },
    ).rename("EVI")
    return image.addBands(evi)


def mask_qa(image):
    qa = image.select("QA_PIXEL")
    mask = qa.bitwiseAnd(1 << 3).eq(0)
    return image.updateMask(mask)

In [13]:
# Landsat 5 (1990-2000)
dataset_l5 = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    .filterDate("1990-01-01", "2000-12-31")
    .filterBounds(aoi_gee)
    .map(mask_qa)
    .map(apply_scale_factors_l5_l7)
    .map(add_evi_l5_l7)
)


In [19]:
def download_image_l5(image, output_dir):
    date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
    filename = f"{date}.tif"
    filepath = os.path.join(output_dir, filename)

    image_evi = image.select("EVI")

    geemap.ee_export_image(
        image_evi,
        filepath,
        scale=250,
        region=aoi_gee,
        file_per_band=False,
        crs="EPSG:4326",
    )
    return filename

In [18]:
output_dir = "../data/landsat_images"
os.makedirs(output_dir, exist_ok=True)

In [16]:
image_list = dataset_l5.toList(dataset_l5.size())
size = image_list.size().getInfo()
print(size)

151


In [20]:
for i in range(size):
    image = ee.Image(image_list.get(i))
    filename = download_image_l5(image, output_dir)

Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-01-15.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-03-04.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-04-21.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-05-07.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-06-08.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-08-27.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-09-12.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsa

In [40]:
# Function to apply scaling factor and mask clouds
def preprocess_modis(image):
    # Mask cloudy pixels using the 'QA' band
    qa = image.select("SummaryQA")  # Quality assurance band
    mask = qa.eq(0)  # Keep pixels with QA = 0 (highest quality)

    # Return the scaled EVI with clouds masked
    return image.updateMask(mask)


# MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m (2001-2023)
dataset_modis = (
    ee.ImageCollection("MODIS/061/MOD13Q1")
    .filterDate("2001-01-01", "2023-12-31")
    .filterBounds(aoi_gee)
    .map(preprocess_modis)
)

In [29]:
output_dir = "../data/modis_images"
os.makedirs(output_dir, exist_ok=True)

In [41]:
image_list = dataset_modis.toList(dataset_modis.size())
size = image_list.size().getInfo()
print(size)

529


In [42]:
def download_image_modis(image, output_dir):
    date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
    filename = f"{date}.tif"
    filepath = os.path.join(output_dir, filename)

    image_evi = image.select("EVI").multiply(0.0001)
    geemap.ee_export_image(
        image_evi,
        filepath,
        scale=250,
        region=aoi_gee,
        file_per_band=False,
        crs="EPSG:4326",
    )
    return filename

In [43]:
for i in range(size):
    image = ee.Image(image_list.get(i))
    filename = download_image_modis(image, output_dir)

Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-01-01.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-01-17.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-02-02.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-02-18.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-03-06.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-03-22.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-04-07.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/modis_images/2001-04

In [23]:
TreeCover = ee.ImageCollection("MODIS/006/MOD44B").first().select("Percent_Tree_Cover")
geemap.ee_export_image(
    TreeCover,
    filename="../data/Percent_Tree_Cover.tif",
    scale=250,
    region=aoi_gee,
    crs="EPSG:4326",
    file_per_band=False,
)

Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/Percent_Tree_Cover.tif
